In [15]:
"""
Module with functions for interacting with Redis
"""
from rediscluster import RedisCluster
import utilprocs

class RedisClient:
    """
    This class is used for reading redis messages
    """    
    def __init__(self, host, stream):
        self.stream = stream
        operandIPs = self.get_ip_by_pod_name(self.get_operand_pod_name())
        startup_nodes = [{"host": host, "port": "6379"}]
        host_port_remap = []
        for ip in operandIPs:
            host_port_remap.append(
                {
                    'from_host': ip,
                    'from_port': 6379,
                    'to_host': host,
                    'to_port': 6379,
                }
            )
        print(host_port_remap)
        self.client = RedisCluster(
            startup_nodes=startup_nodes,
            decode_responses=True,
            host_port_remap=host_port_remap
        )

        
    def get_latest_notification(self):
        """
        Reads the lates redis message
        """
        return self.client.xrevrange(self.stream, count=1)[0][1]['notification']

    def validate_latest_notification(self, action_id, backup_manager_id, action):
        """
        Validates a restore notification message to make sure it's in the
        correct format

        :param action_id: the expected action id of the message
        :param backup_manager_id: the expected backup manager id of the message
        :param action: RESTORE or CREATE_BACKUP
        """

        expected_message = f'Notification [version=1.0.0, action={action}, actionId={action_id}, backupManagerId={backup_manager_id}, status=COMPLETED]'

        actural_message = self.get_latest_notification()

        error = 'Invalid value for notification: Expected message: ' \
            '"{}", Actual message: "{}"'

        assert expected_message == actural_message , error.format(expected_message, actural_message)

    def close(self):
        """
        Closes the redis client
        """
        self.client.close()

    def get_operand_pod_name(self):
        response = utilprocs.execute_command("kubectl -n eyicwux-test1 get po -l "
                                      "app.kubernetes.io/name=eric-data-key-value-database-rd-operand -o jsonpath={..metadata.name}" , log_command=False)
        return response.split();


    def get_ip_by_pod_name(self, pods):
        ips = []
        for pod in pods:
            res = utilprocs.execute_command("kubectl -n eyicwux-test1 get pods {} -o=jsonpath={{.status.podIP}}".format(pod), log_command=False)
            ips.append(res)
        return ips


In [16]:
rc = RedisClient("10.120.217.249", "bro-notification")

eric-data-key-value-database-rd-operand-9rbvd eric-data-key-value-database-rd-operand-whvx4
192.168.37.234
192.168.129.201
[{'from_host': '192.168.37.234', 'from_port': 6379, 'to_host': '10.120.217.249', 'to_port': 6379}, {'from_host': '192.168.129.201', 'from_port': 6379, 'to_host': '10.120.217.249', 'to_port': 6379}]


In [18]:
rc.get_latest_notification()

'Notification [version=1.0.0, action=CREATE_BACKUP, actionId=30576, backupManagerId=configuration-data, status=COMPLETED]'

In [17]:
rc.client.ping()

{'10.120.217.249:6379': True}

In [19]:
rc.validate_latest_notification(30576, 'configuration-data', 'CREATE_BACKUP')

In [ ]:
rc.